In [12]:
import pandas as pd
import numpy as np
import math
from sklearn.cluster import KMeans
import sklearn.preprocessing as preprocessing


In [24]:
data = pd.read_csv('Indicators.csv')
data.shape

(5656458, 6)

In [4]:
rel_columns_8 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Birth rate, crude (per 1,000 people)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [5]:
# 여기서부터 관련도 높은 feature 뽑는 것
target_id = 'Adolescent fertility rate (births per 1,000 women ages 15-19)';

target = data[data['IndicatorName']==target_id]

In [25]:
merged = data[data['IndicatorName']==target_id]

for i in rel_columns_8:
    target = data[data['IndicatorName']==i]
    target.rename(columns={"Value": i}, inplace = True)
    target=target.drop(columns=['IndicatorCode', 'IndicatorName', 'CountryName'])
#    print(target.head())
    merged_bef = merged
    merged = pd.merge(merged, target)
    if len(merged) < 500:
        merged = merged_bef
#    print(merged.head())

merged

c:\users\parksoyoung\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5)
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,69.020564,69.020564,35.138300,44.576275,39.293392,49.667217,53.673302
1,Caribbean small states,CSS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,90.664634,90.664634,25.257613,64.583374,66.755806,62.447144,42.518994
2,Euro area,EMU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,14.604634,14.604634,11.720194,79.480003,80.562958,78.446426,19.434867
3,European Union,EUU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,22.283391,22.283391,12.375231,80.662552,81.711151,79.662552,22.427980
4,High income,HIC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,33.709503,33.709503,14.585758,81.518730,82.150581,80.926880,23.921785
5,High income: OECD,OEC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,26.662798,26.662798,13.652285,83.069412,83.782578,82.389725,18.861448
6,Least developed countries: UN classification,LDC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,135.511948,135.511948,42.473446,15.394602,12.155324,18.571140,75.306228
7,Middle East & North Africa (all income levels),MEA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,68.680675,68.680675,33.544956,49.202000,44.690079,53.519825,48.646734
8,Middle East & North Africa (developing only),MNA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,70.065273,70.065273,33.850139,49.028133,44.022461,53.780590,48.743728
9,North America,NAC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,53.848894,53.848894,16.529283,83.866615,84.452255,83.308624,16.563037


In [28]:
merged['grade'] = merged['Value']/50
for i in range(len(merged)):
    merged.loc[i, 'grade'] = math.ceil(merged.loc[i, 'grade'])
merged.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),grade
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,69.020564,69.020564,35.138300,44.576275,39.293392,49.667217,53.673302,2.0
1,Caribbean small states,CSS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,90.664634,90.664634,25.257613,64.583374,66.755806,62.447144,42.518994,2.0
2,Euro area,EMU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,14.604634,14.604634,11.720194,79.480003,80.562958,78.446426,19.434867,1.0
3,European Union,EUU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,22.283391,22.283391,12.375231,80.662552,81.711151,79.662552,22.427980,1.0
4,High income,HIC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,33.709503,33.709503,14.585758,81.518730,82.150581,80.926880,23.921785,1.0


In [22]:
# label encode categorical data (array)
def labelEncode_arr(y):
    '''
    label encoding array
    return value: label encoded array
    '''
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y= le.transform(y)
    return y

In [29]:
merged['CountryName']= labelEncode_arr(merged['CountryName'])
merged['IndicatorName']= labelEncode_arr(merged['IndicatorName'])

In [14]:
def calcAccuracy(x,y):
    '''
    print accuracy of prediction
    '''
    correct=0
    for i in range(len(x)):
        predict_me= np.array(x[i].astype(float))
        predict_me = predict_me.reshape(-1, len(predict_me)) 
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
    print(correct/len(x))

In [31]:
merged.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),grade
0,5,ARB,0,SP.ADO.TFRT,1990,69.020564,69.020564,35.138300,44.576275,39.293392,49.667217,53.673302,2.0
1,28,CSS,0,SP.ADO.TFRT,1990,90.664634,90.664634,25.257613,64.583374,66.755806,62.447144,42.518994,2.0
2,50,EMU,0,SP.ADO.TFRT,1990,14.604634,14.604634,11.720194,79.480003,80.562958,78.446426,19.434867,1.0
3,53,EUU,0,SP.ADO.TFRT,1990,22.283391,22.283391,12.375231,80.662552,81.711151,79.662552,22.427980,1.0
4,67,HIC,0,SP.ADO.TFRT,1990,33.709503,33.709503,14.585758,81.518730,82.150581,80.926880,23.921785,1.0


In [32]:
merged=merged.drop(columns=['IndicatorCode', 'CountryCode'])

In [36]:
merged.head()

,CountryName,IndicatorName,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),grade
0,5,0,1990,69.020564,69.020564,35.138300,44.576275,39.293392,49.667217,53.673302,2.0
1,28,0,1990,90.664634,90.664634,25.257613,64.583374,66.755806,62.447144,42.518994,2.0
2,50,0,1990,14.604634,14.604634,11.720194,79.480003,80.562958,78.446426,19.434867,1.0
3,53,0,1990,22.283391,22.283391,12.375231,80.662552,81.711151,79.662552,22.427980,1.0
4,67,0,1990,33.709503,33.709503,14.585758,81.518730,82.150581,80.926880,23.921785,1.0


In [39]:
X= np.array(merged.drop(['grade'], 1).astype(float))
y=merged['grade']

In [43]:
# trial 1
kmeans= KMeans(n_clusters=5, max_iter=600, algorithm='auto')
kmeans.fit(X)
calcAccuracy(X,y)

0.38241415192507805
